In [3]:
import pandas as pd
import re
import json
import xml.etree.ElementTree as xet
import requests as req
import time
import glob
import csv
import regex
import tqdm
import pytz
import numpy as np
from bs4 import BeautifulSoup

In [4]:
# min_id = 2979000
# max_id = 3524639
# ids = np.arange(min_id, max_id + 1).astype(str)

In [5]:
min_id = 3658119
max_id = 4117093
ids = np.arange(min_id, max_id + 1).astype(str)

In [6]:
df = pd.DataFrame({'url': np.array(list(map(lambda x: "https://www.kp.ru/online/news/" + x + "/", ids)))})
df.index = np.arange(min_id, max_id + 1)
df.to_json('news_kp.json')
df.shape

(458975, 1)

# Parsing

In [15]:
def parse_news(text):
    soup = BeautifulSoup(text, features='lxml')
    
    news_text = ''
    news_body = soup.find(name='div', attrs={'class': 'js-mediator-article'})
    
    if news_body:
        scripts = news_body.find_all(name=['script'])
        if scripts:
            [script.decompose() for script in scripts]
        
        news_ps = news_body.find_all(name='p')
        if news_ps:
            news_text += " " + " ".join([news_p.get_text(" ", strip=True) for news_p in news_ps])
        news_text = re.sub(r'[\n\r]+', ' ', news_text)

    lastmod_elem = soup.find(name="meta", attrs={'name': "mediator_published_time"})
    lastmod = re.sub(r'[\n\r]+', ' ', lastmod_elem.get("content")) if lastmod_elem else ''
    if lastmod:
        lastmod = pd.to_datetime(lastmod, utc=True).strftime("%Y-%m-%dT%H:%M:%SZ")
    
    tags_elem = soup.find(name="meta", attrs={'name': "keywords"})
    tags = re.sub(r'[\n\r]+', ' ', tags_elem.get("content")) if tags_elem else ''
    
    description_elem = soup.find(name="meta", attrs={'name': "description"})
    description = re.sub(r'[\n\r]+|<[^>]+>', ' ', description_elem.get('content')) if description_elem else ''
        
    return {'lastmod': lastmod, 'news_text': news_text, 'tags': tags, 'description': description}

In [23]:
df.iloc[0].url

'https://www.kp.ru/online/news/2979000/'

In [24]:
parse_news(req.get(df.iloc[0].url).content)

{'lastmod': '2017-12-30T17:15:31+0300',
 'news_text': '  Мэр Сергей Иванов и председатель городского Совета депутатов Игорь Тиньков поздравили жителей Липецка в Новым годом и Рождеством. – Провожать 2017-ый немного грустно: год был непростым, но интересным, насыщенным яркими событиями, – говорится в поздравлении, – нам удалось немало сделать для развития Липецка. И, конечно, все перемены к лучшему стали возможными благодаря активному участию в судьбе родного города его жителей, за что огромное спасибо землякам. Первые лица города рассказали о том, что удалось сделать в уходящем году: масштабный ремонт дорог, обновление пассажирского транспорта, строительство домов и школ, благоустройство улиц и парков, и многое другое. «Новогодние праздники всегда проникнуты атмосферой добра и тепла, ожиданием чуда. Позвольте от всего сердца пожелать вам и вашим близким здоровья, любви, счастья, много солнечных дней! Пусть исполнится все задуманное, все ваши самые заветные желания! С праздниками, дорог